# Lesson 1: Building blocks

Welcome to this course! Just a reminder that LLMs may not always return the same text. As a result, the outputs you see when running the notebook may vary somewhat from those you see in the film. This applies to all of the notebooks.
Enjoy the course!

## Language model

1. Text LLMs: string -> string
2. Chat models: list of messages -> single message output

In [1]:
import {ChatOllama} from '@langchain/ollama';
import { HumanMessage } from '@langchain/core/messages';
import { ChatPromptTemplate } from "@langchain/core/prompts";

In [2]:
import {ChatOllama} from '@langchain/ollama';
const model = new ChatOllama({
    model:  "llama3.1:latest",
    baseUrl: "http://localhost:11434"
});


## Prompt template

In [3]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromTemplate(
    `What are three good names for a company that makes {product}?`
)

In [4]:
await prompt.format({
    product: "colorful socks"
});

"Human: What are three good names for a company that makes colorful socks?"

In [5]:
await prompt.formatMessages({
    product: "colorful socks"
});

[
  HumanMessage {
    "content": "What are three good names for a company that makes colorful socks?",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]

In [6]:
import { 
  SystemMessagePromptTemplate, 
  HumanMessagePromptTemplate 
} from "@langchain/core/prompts";

const promptFromMessages = ChatPromptTemplate.fromMessages([
  SystemMessagePromptTemplate.fromTemplate(
    "{system_info}"
  ),
  HumanMessagePromptTemplate.fromTemplate(
    "Write a resume for the {role} at {company_name}?"
  )
]);

await promptFromMessages.formatMessages({
  system_info: "You are an expert at writing reesumes for job roles.",
  role: "Data scientist",
  company_name: "Meta"
});

[
  SystemMessage {
    "content": "You are an expert at writing reesumes for job roles.",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "Write a resume for the Data scientist at Meta?",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]

In [7]:
// const promptFromMessages = ChatPromptTemplate.fromMessages([
//   ["system", "You are an expert at picking company names."],
//   ["human", "What are three good names for a company that makes {product}?"]
// ]);

// await promptFromMessages.formatMessages({
//   product: "shiny objects"
// });

# LangChain Expression Language (LCEL)

In [8]:
const chain = promptFromMessages.pipe(model);

In [9]:
await chain.invoke({
    system_info: "You are an expert at writing reesumes for job roles.",
    role: "Data scientist",
    company_name: "Meta"
});

AIMessage {
  "content": "Here's a sample resume for a Data Scientist position at Meta:\n\n**Your Name**\n**Contact Information**\n\n**Summary:**\nHighly motivated and detail-driven data scientist with 5+ years of experience in building predictive models, developing machine learning algorithms, and driving business growth through data-driven insights. Proven track record of working on large-scale datasets, collaborating with cross-functional teams, and delivering high-impact results.\n\n**Technical Skills:**\n\n* Programming languages: Python, R, SQL\n* Machine learning frameworks: TensorFlow, PyTorch, Scikit-learn\n* Data visualization tools: Tableau, Power BI, D3.js\n* Operating Systems: Windows, Linux\n* Familiarity with distributed computing environments (Hadoop, Spark)\n\n**Professional Experience:**\n\n**Senior Data Scientist**\n[Previous Company], [Location]\n[Dates]\n\n* Led a team of data scientists to develop predictive models for user engagement and churn prediction, resulti

## Output parser

In [10]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const outputParser = new StringOutputParser();

const nameGenerationChain = promptFromMessages.pipe(model).pipe(outputParser);

In [11]:
await nameGenerationChain.invoke({
    system_info: "You are an expert at writing reesumes for job roles.",
    role: "Data Scientist",
    company_name:"Meta"
});

"Here's a sample resume for a Data Scientist position at Meta:\n" +
  "\n" +
  "**Name:** Emily J. Chen\n" +
  "\n" +
  "**Contact Information:**\n" +
  "\n" +
  "* Email: [emily.chen@email.com](mailto:emily.chen@email.com)\n" +
  "* Phone: 555-123-4567\n" +
  "* LinkedIn: linkedin.com/in/emilyjchen\n" +
  "\n" +
  "**Summary:**\n" +
  "Highly motivated and detail-driven data scientist with 5+ years of experience in developing and implementing predictive models, statistical analysis, and machine learning algorithms to drive business growth and inform product development. Proven track record of successfully collaborating with cross-functional teams to deliver high-impact projects.\n" +
  "\n" +
  "**Technical Skills:**\n" +
  "\n" +
  "* Programming languages: Python, R, SQL\n" +
  "* Machine Learning frameworks: TensorFlow, PyTorch, Scikit-Learn\n" +
  "* Data platforms: AWS S3, GCP BigQuery, Apache Spark\n" +
  "* Statistical analysis and visualization tools: NumPy, Pandas, Matplotlib

In [12]:
import { RunnableSequence } from "@langchain/core/runnables";

const nameGenerationChain = RunnableSequence.from([
    promptFromMessages,
    model,
    outputParser
])

## Streaming

In [13]:
const stream = await nameGenerationChain.stream({
  system_info: "You are an expert at writing reesumes for job roles.",
  role: "Data Scientist",
  company_name:"Meta"
});

for await (const chunk of stream) {
    console.log(chunk);
}

Here
's
 a
 sample
 resume
 for
 a
 Data
 Scientist
 position
 at
 Meta
:


**
Your
 Name
**

**
Contact
 Information
**


**
Summary
:
**

High
ly
 motivated
 and
 detail
-oriented
 data
 scientist
 with
 
5
+
 years
 of
 experience
 in
 developing
 and
 deploying
 predictive
 models
,
 visual
izing
 complex
 data
,
 and
 communicating
 insights
 to
 both
 technical
 and
 non
-
technical
 stakeholders
.
 Pro
ven
 track
 record
 of
 driving
 business
 growth
 through
 data
-driven
 decision
 making
.


**
Technical
 Skills
:
**


*
 Programming
 languages
:
 Python
,
 R
,
 SQL


*
 Data
 science
 tools
:
 TensorFlow
,
 Py
T
orch
,
 Sc
ikit
-
learn
,
 K
eras


*
 Data
 visualization
:
 Table
au
,
 Power
 BI
,
 Mat
plotlib
,
 Se
ab
orn


*
 Machine
 learning
 algorithms
:
 Linear
 regression
,
 Decision
 trees
,
 Random
 forests
,
 Neural
 networks


*
 Experience
 with
 big
 data
 processing
:
 H
adoop
,
 Spark


*
 Fam
iliar
ity
 with
 cloud
 platforms
:
 AWS
,
 G
CP



**
Professional